In [1]:
import functools
import itertools
import logging
import math
import os
import pickle
import sys
from pathlib import Path

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import seaborn as sns
import yaml
from itertools import product

%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

sns.set_context("poster")
sns.set(rc={"figure.figsize": (16, 12.0)})
sns.set_style("whitegrid")

import numpy as np
import pandas as pd
import torch.nn.functional as F

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [2]:
from lda4rec.datasets import Interactions, DataLoader, random_train_test_split
from lda4rec.evaluations import mrr_score, precision_recall_score, summary
from lda4rec.estimators import MFEst, PopEst, LDA4RecEst, SNMFEst
from lda4rec.utils import process_ids, cmp_ranks

In [3]:
import pyro
import pyro.distributions as dist
import pyro.optim as optim
import torch
from pyro.distributions import constraints
from pyro.infer import SVI, Predictive, Trace_ELBO, TraceEnum_ELBO, config_enumerate

In [4]:
import neptune.new as neptune
# init dummy neptune to avoid problems with logging
neptune.init(mode='offline') 

offline/93463181-7f81-4d37-8cb9-5e1d9a01a540


In [5]:
from icecream import ic, install

install()
# configure icecream
def ic_str(obj):
    if hasattr(obj, "shape"):
        return f"{obj} "  #
    else:
        return str(obj)

In [6]:
ic.configureOutput(argToStringFunction=ic_str)

## Results exported from Neptune.AI

In [7]:
df = pd.read_csv("../results/experiments.csv")

In [8]:
df.rename(columns=lambda x: x.split("/")[-1].split(" ")[0], inplace=True)

In [9]:
metrics = {f"{m}_{s}": np.mean for m, s in product(["mrr", "prec", "recall"], ["train", "valid", "test"])}
# metrics['Id'] = "count"
idx_order = ["PopEst", "NMFEst", "SNMFEst", "MFEst", "LDA4RecEst"]

In [10]:
df.loc[(df["dataset"] == "movielens-1m")].groupby(["estimator", "embedding_dim"], dropna=False).agg(metrics).reindex(index=idx_order, level=0)

mrr_train  mrr_valid  mrr_test  prec_train  \
estimator embedding_dim                                               
PopEst    NaN             0.023706   0.032858  0.033488    0.339636   
NMFEst    4.0             0.022534   0.031129  0.031757    0.338088   
          8.0             0.023526   0.032614  0.033156    0.339216   
          16.0            0.023637   0.032580  0.033320    0.339205   
          32.0            0.023615   0.032365  0.033103    0.338433   
SNMFEst   4.0             0.029322   0.043518  0.044251    0.412130   
          8.0             0.032732   0.051700  0.051630    0.460662   
          16.0            0.037630   0.063475  0.062563    0.519407   
          32.0            0.040289   0.069753  0.067168    0.548780   
MFEst     4.0             0.030659   0.047478  0.048307    0.436071   
          8.0             0.035171   0.058181  0.058181    0.492817   
          16.0            0.038462   0.064867  0.063977    0.528877   
          32.0            0.041237   0.068536  0.066223    0.553598   

                         prec_valid  prec_test  recall_train  recall_valid  \
estimator embedding_dim                                                      
PopEst    NaN              0.032154   0.033084      0.045557      0.064532   
NMFEst    4.0              0.032115   0.033138      0.045410      0.064410   
          8.0              0.032139   0.033176      0.045510      0.064685   
          16.0             0.032192   0.033259      0.045494      0.064726   
          32.0             0.031941   0.032782      0.045527      0.064019   
SNMFEst   4.0              0.043205   0.043614      0.057985      0.089223   
          8.0              0.051649   0.051628      0.066209      0.108178   
          16.0             0.062570   0.061028      0.078043      0.133402   
          32.0             0.067481   0.065552      0.084599      0.146745   
MFEst     4.0              0.047080   0.046846      0.061627      0.096663   
          8.0              0.057111   0.055825      0.072491      0.121395   
          16.0             0.064393   0.062587      0.080149      0.138847   
          32.0             0.066649   0.065004      0.086069      0.145797   

                         recall_test  
estimator embedding_dim               
PopEst    NaN               0.065908  
NMFEst    4.0               0.066450  
          8.0               0.066027  
          16.0              0.066263  
          32.0              0.065677  
SNMFEst   4.0               0.090498  
          8.0               0.107532  
          16.0              0.129522  
          32.0              0.141108  
MFEst     4.0               0.097700  
          8.0               0.118196  
          16.0              0.133671  
          32.0              0.139404

In [11]:
df.loc[(df["dataset"] == "goodbooks")].groupby(["estimator", "embedding_dim"], dropna=False).agg(metrics).reindex(index=idx_order, level=0)

mrr_train  mrr_valid  mrr_test  prec_train  \
estimator embedding_dim                                               
PopEst    NaN             0.015454   0.023697  0.023918    0.297855   
NMFEst    4.0             0.009325   0.013302  0.013347    0.234443   
          8.0             0.008760   0.012447  0.012470    0.229933   
          16.0            0.012145   0.018133  0.018254    0.276424   
          32.0            0.017977   0.028189  0.028376    0.304750   
SNMFEst   4.0             0.020552   0.035483  0.035509    0.401228   
          8.0             0.023429   0.041675  0.042030    0.448016   
          16.0            0.026163   0.048722  0.048844    0.495876   
          32.0            0.028218   0.054817  0.054590    0.531036   
MFEst     4.0             0.021690   0.037178  0.037361    0.421354   
          8.0             0.024290   0.044041  0.044052    0.463748   
          16.0            0.026592   0.050465  0.050721    0.504217   
          32.0            0.028436   0.055303  0.055351    0.535559   

                         prec_valid  prec_test  recall_train  recall_valid  \
estimator embedding_dim                                                      
PopEst    NaN              0.026989   0.027079      0.029935      0.047702   
NMFEst    4.0              0.019982   0.019691      0.023328      0.034696   
          8.0              0.018830   0.018680      0.022863      0.032696   
          16.0             0.024649   0.024602      0.027715      0.043186   
          32.0             0.027921   0.027900      0.030696      0.049326   
SNMFEst   4.0              0.039865   0.039857      0.040393      0.070296   
          8.0              0.046693   0.046743      0.045997      0.083610   
          16.0             0.054867   0.054872      0.051690      0.099578   
          32.0             0.061892   0.061933      0.055600      0.112821   
MFEst     4.0              0.042074   0.042265      0.042876      0.074786   
          8.0              0.048954   0.049005      0.048062      0.088400   
          16.0             0.056346   0.056614      0.052566      0.102128   
          32.0             0.062390   0.062411      0.056040      0.113742   

                         recall_test  
estimator embedding_dim               
PopEst    NaN               0.047867  
NMFEst    4.0               0.034357  
          8.0               0.032619  
          16.0              0.043182  
          32.0              0.049352  
SNMFEst   4.0               0.070744  
          8.0               0.084337  
          16.0              0.099674  
          32.0              0.112936  
MFEst     4.0               0.075502  
          8.0               0.088667  
          16.0              0.103009  
          32.0              0.113782

In [12]:
df.loc[df["dataset"] == "movielens-100k"].groupby(["estimator", "embedding_dim",], dropna=False).agg(metrics).reindex(index=idx_order, level=0)

mrr_train  mrr_valid  mrr_test  prec_train  \
estimator  embedding_dim                                               
PopEst     NaN             0.026137   0.042217  0.037069    0.328525   
MFEst      4.0             0.030171   0.054383  0.049304    0.370984   
           8.0             0.031930   0.054979  0.047378    0.387869   
           16.0            0.035087   0.056758  0.046867    0.436557   
LDA4RecEst 4.0             0.034830   0.056307  0.066236    0.426393   
           8.0             0.034740   0.043342  0.053524    0.419180   
           16.0            0.038174   0.050153  0.058407    0.464426   

                          prec_valid  prec_test  recall_train  recall_valid  \
estimator  embedding_dim                                                      
PopEst     NaN              0.026799   0.031148      0.054219      0.069386   
MFEst      4.0              0.040108   0.041530      0.063030      0.107724   
           8.0              0.040468   0.042987      0.067500      0.113402   
           16.0             0.042446   0.045173      0.072697      0.115023   
LDA4RecEst 4.0              0.045863   0.048816      0.076604      0.133678   
           8.0              0.046403   0.045902      0.072968      0.115465   
           16.0             0.044245   0.046995      0.082597      0.106134   

                          recall_test  
estimator  embedding_dim               
PopEst     NaN               0.072635  
MFEst      4.0               0.099996  
           8.0               0.110723  
           16.0              0.111719  
LDA4RecEst 4.0               0.130728  
           8.0               0.119682  
           16.0              0.125783

## Extract test scores by selecting the best validation score

In [14]:
def get_best_valid(df):
    """Select test score by best validation score"""
    results = {}
    for metric in ["mrr", "prec", "recall"]:
        best = df[f"{metric}_valid"].max()
        results[metric] = df[f"{metric}_test"].loc[df[f"{metric}_valid"] == best].values[0]
    return pd.Series(results)

In [17]:
(df.loc[df["dataset"].isin(["movielens-1m", "goodbooks"])]
     .groupby(["estimator", "embedding_dim", "dataset"], dropna=False)
     .apply(get_best_valid)
     .reset_index()
     .melt(id_vars=["estimator", "embedding_dim", "dataset"], value_vars=["mrr", "prec", "recall"], var_name="metric")
     .sort_values(["estimator", "embedding_dim", "dataset"])
     .pivot(index=["estimator", "embedding_dim"], values="value", columns=["dataset", "metric"])
     .reindex(index=idx_order, level=0)
     )

dataset                 goodbooks                     movielens-1m            \
metric                        mrr      prec    recall          mrr      prec   
estimator embedding_dim                                                        
PopEst    NaN            0.023918  0.027079  0.047867     0.033488  0.033084   
NMFEst    4.0            0.014388  0.022056  0.038420     0.032927  0.033031   
          8.0            0.015121  0.019261  0.033310     0.033445  0.033191   
          16.0           0.019945  0.026436  0.046623     0.033461  0.033351   
          32.0           0.028766  0.028268  0.050012     0.033223  0.033155   
SNMFEst   4.0            0.036186  0.040912  0.072584     0.046642  0.046211   
          8.0            0.042435  0.047185  0.085326     0.053639  0.052028   
          16.0           0.049671  0.055800  0.101652     0.062695  0.061526   
          32.0           0.055048  0.062215  0.113179     0.068511  0.066453   
MFEst     4.0            0.038901  0.044045  0.079124     0.050495  0.048702   
          8.0            0.044683  0.049835  0.090115     0.058240  0.057044   
          16.0           0.050875  0.057127  0.103747     0.063849  0.062131   
          32.0           0.056080  0.062841  0.114629     0.064506  0.064888   

dataset                            
metric                     recall  
estimator embedding_dim            
PopEst    NaN            0.065908  
NMFEst    4.0            0.066281  
          8.0            0.066516  
          16.0           0.066191  
          32.0           0.064652  
SNMFEst   4.0            0.097219  
          8.0            0.108542  
          16.0           0.130733  
          32.0           0.141667  
MFEst     4.0            0.103090  
          8.0            0.119924  
          16.0           0.131973  
          32.0           0.138600

In [16]:
df.loc[(df["dataset"] == "movielens-100k")].groupby(["estimator", "embedding_dim"], dropna=False).apply(get_best_valid).reindex(index=idx_order, level=0)

mrr      prec    recall
estimator  embedding_dim                              
PopEst     NaN            0.037069  0.031148  0.072635
MFEst      4.0            0.049304  0.041530  0.099996
           8.0            0.047378  0.042987  0.110723
           16.0           0.046867  0.045173  0.111719
LDA4RecEst 4.0            0.066236  0.048816  0.130728
           8.0            0.053524  0.045902  0.119682
           16.0           0.058407  0.046995  0.125783